<b> Recurrent Neural Network Basics </b>

<br/>
This notebook is for basic understanding of RNNs. Most of the content are borrowed (as it is from [1] and others in ref.) from the references. I fully acknowledge the original authors for their resource.

<br/>
The idea behind RNNs is to make use of sequential information. In a traditional neural network we assume that all inputs (and outputs) are independent of each other. But for many tasks that’s a very bad idea. If you want to predict the next word in a sentence you better know which words came before it. RNNs are called recurrent because they perform the same task for every element of a sequence, with the output being depended on the previous computations.

<br/>
Another way to think about RNNs is that they have a “memory” which captures information about what has been calculated so far. In theory RNNs can make use of information in arbitrarily long sequences, but in practice they are limited to looking back only a few steps (more on this later). Here is what a typical RNN looks like:
<br/>
<style>
figure {
    display: table;
    border: 1px solid black;
    margin: auto;
}
</style>
<figure>
    <img src = "images/rnn_basic.jpg">
    <figcaption>A recurrent neural network and the unfolding in time of the computation involved in its forward computation. Source: Nature</figcaption>
</figure>
<br/>
The left image shows the basic rnn and the right one shows the unfolded version of the same network.
By unrolling we simply mean that we write out the network for the complete sequence. For example, if the sequence we care about is a sentence of 5 words, the network would be unrolled into a 5-layer neural network, one layer for each word. The formulas that govern the computation happening in a RNN are as follows:
<ul>
<li> $x_t$ is the input at time step t. For example, $x_1$ could be a one-hot vector corresponding to the second word of a sentence.
<li> $s_t$ is the hidden state at time step t. It’s the “memory” of the network.  s_t is calculated based on the previous hidden state and the input at the current step: $s_t=f(Ux_t + Ws_{t-1})$. The function $f$ usually is a nonlinearity such as tanh or ReLU.  $s_{-1}$, which is required to calculate the first hidden state, is typically initialized to all zeroes.
<li> $o_t$ is the output at step $t$. For example, if we wanted to predict the next word in a sentence, it would be a vector of probabilities across our vocabulary. $o_t = \mathrm{softmax}(Vs_t)$.
</ul>

<br/>
There are a few things to note here:
<ul>
<li>The hidden state $s_t$ can be taken as the memory of the network. $s_t$ captures information about what happened in all the previous time steps. The output at step $o_t$ is calculated solely based on the memory at time $t$. As briefly mentioned above, it’s a bit more complicated  in practice because $s_t$ typically can’t capture information from too many time steps ago.
<li>Unlike a traditional deep neural network, which uses different parameters at each layer, a RNN shares the same parameters ($U, V, W$ above) across all steps. This reflects the fact that we are performing the same task at each step, just with different inputs. This greatly reduces the total number of parameters we need to learn.
<li>The above diagram has outputs at each time step, but depending on the task this may not be necessary. For example, when predicting the sentiment of a sentence we may only care about the final output, not the sentiment after each word. Similarly, we may not need inputs at each time step. The main feature of an RNN is its hidden state, which captures some information about a sequence.

<br/>
The RNNs have been successfully used in many sequence modeling problems, such as:
<ol>
<li> Language Modeling and Text Generation
<ol>
<li> <a href="http://www.fit.vutbr.cz/research/groups/speech/publi/2010/mikolov_interspeech2010_IS100722.pdf"> RNN based language model </a>
<li> <a href="http://www.fit.vutbr.cz/research/groups/speech/publi/2011/mikolov_icassp2011_5528.pdf"> Extensions of RNN based language model </a>
<li> <a href="http://machinelearning.wustl.edu/mlpapers/paper_files/ICML2011Sutskever_524.pdf">  Generating Text with RNNs</a>
</ol>
<li> Machine Translation
<ol>
<li> <a href="http://www.aclweb.org/anthology/P14-1140.pdf"> A Recursive Recurrent Neural Network for Statistical Machine Translation</a>
<li> <a href="http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf"> Sequence to Sequence Learning with Neural Networks</a>
<li> <a href="http://research.microsoft.com/en-us/um/people/gzweig/Pubs/EMNLP2013RNNMT.pdf"> Joint Language and Translation Modeling with Recurrent Neural Networks </a>
</ol>
<li> Speech Recognition
<li> Image caption generation
<ol>
<li> http://cs.stanford.edu/people/karpathy/deepimagesent/
</ol>
</ol>
<br/>

<b> Training RNN </b>

<br/>
Training a RNN is similar to training a traditional Neural Network. We also use the backpropagation algorithm, but with a little twist. Because the parameters are shared by all time steps in the network, the gradient at each output depends not only on the calculations of the current time step, but also the previous time steps. For example, in order to calculate the gradient at $t=4$ we would need to backpropagate 3 steps and sum up the gradients. This is called Backpropagation Through Time (BPTT). We just need to be aware of the fact that vanilla RNNs trained with BPTT have difficulties learning long-term dependencies (e.g. dependencies between steps that are far apart) due to what is called the vanishing/exploding gradient problem. There exists some machinery to deal with these problems, and certain types of RNNs (like LSTMs) were specifically designed to get around them.

<br/>
<b> Bidirectional RNN</b>
are based on the idea that the output at time t may not only depend on the previous elements in the sequence, but also future elements. For example, to predict a missing word in a sequence you want to look at both the left and the right context. Bidirectional RNNs are quite simple. They are just two RNNs stacked on top of each other. The output is then computed based on the hidden state of both RNNs.
<img src="images/bidirectional-rnn.png">

<br/>
<b> Deep Bidirectional RNN</b>
are similar to Bidirectional RNNs, only that we now have multiple layers per time step. In practice this gives us a higher learning capacity (but we also need a lot of training data).
<img src="images/deep-bidirectional-rnn.png"/>

<br/>
<b> Language Modeling with RNN</b>
<br/>

Our goal is to build a Language Model using a Recurrent Neural Network. Here’s what that means. Let’s say we have sentence of m words. A language model allows us to predict the probability of observing the sentence (in a given dataset) as:

\begin{aligned}  P(w_1,...,w_m) = \prod_{i=1}^{m} P(w_i \mid w_1,..., w_{i-1})  \end{aligned}
<br/>
In words, the probability of a sentence is the product of probabilities of each word given the words that came before it. So, the probability of the sentence “He went to buy some chocolate” would be the probability of “chocolate” given “He went to buy some”, multiplied by the probability of “some” given “He went to buy”, and so on.
<br/>
Because we can predict the probability of a word given the preceding words, we are able to generate new text. It’s a generative model. Given an existing sequence of words we sample a next word from the predicted probabilities, and repeat the process until we have a full sentence. <a href="http://karpathy.github.io/2015/05/21/rnn-effectiveness/">Andrej Karparthy has a great post </a> that demonstrates what language models are capable of. His models are trained on single characters as opposed to full words, and can generate anything from Shakespeare to Linux Code.

<br/>
Note that in the above equation the probability of each word is conditioned on all previous words. In practice, many models have a hard time representing such long-term dependencies due to computational or memory constraints. They are typically limited to looking at only a few of the previous words. RNNs can, in theory, capture such long-term dependencies, but in practice it’s a bit more complex. 

<br/>
<b> Data Gathering and Preprocessing </b>
<br/>
To train our language model we need text to learn from. Fortunately we don’t need any labels to train a language model, just raw text. I downloaded 15,000 longish reddit comments from a dataset available on <a href="https://bigquery.cloud.google.com/table/fh-bigquery:reddit_comments.2015_08">Google’s BigQuery </a>. Text generated by our model will sound like reddit commenters (hopefully)! But as with most Machine Learning projects we first need to do some pre-processing to get our data into the right format.

<br/>
<b> Step 1 : Text Tokenization</b>
<br/>
We need to tokenize our comments into sentences, and sentences into words. We could just split each of the comments by spaces, but that wouldn’t handle punctuation properly. The sentence “He left!” should be 3 tokens: “He”, “left”, “!”. We’ll use NLTK’s word_tokenize and sent_tokenize methods, which do most of the hard work for us.


<br/>
<b> Step 2 : Infrequent word removal</b>
<br/>
In our code we limit our vocabulary to the vocabulary_size most common words (which I set to 8000, but feel free to change it). We replace all words not included in our vocabulary by UNKNOWN_TOKEN. For example, if we don’t include the word “nonlinearities” in our vocabulary, the sentence “nonlineraties are important in neural networks” becomes “UNKNOWN_TOKEN are important in Neural Networks”. The word UNKNOWN_TOKEN will become part of our vocabulary and we will predict it just like any other word. When we generate new text we can replace UNKNOWN_TOKEN again, for example by taking a randomly sampled word not in our vocabulary, or we could just generate sentences until we get one that doesn’t contain an unknown token.


<br/>
<b> Step 3 : PREPEND SPECIAL START AND END TOKENS</b>
<br/>
We also want to learn which words tend start and end a sentence. To do this we prepend a special SENTENCE_START token, and append a special SENTENCE_END token to each sentence. This allows us to ask: Given that the first token is SENTENCE_START, what is the likely next word (the actual first word of the sentence)?


<br/>
<b> Step 4 :  BUILD TRAINING DATA MATRICES</b>
<br/>
The input to our Recurrent Neural Networks are vectors, not strings. So we create a mapping between words and indices, index_to_word, and word_to_index. For example,  the word “friendly” may be at index 2001. A training example x may look like [0, 179, 341, 416], where 0 corresponds to SENTENCE_START. The corresponding label y would be [179, 341, 416, 1]. Remember that our goal is to predict the next word, so y is just the x vector shifted by one position with the last element being the SENTENCE_END token. In other words, the correct prediction for word 179 above would be 341, the actual next word.

In [1]:
#Now lets read the data from the csv file (make sure that you download the data file and put inside data folder)
#get the libraries
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
import os
import time
from datetime import datetime
from rnn_utils import *

In [2]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
 
# Read the data and append SENTENCE_START and SENTENCE_END tokens
print("Reading CSV file...")
with open('data/reddit-comments-2015-08.csv' ,encoding="utf8") as f:
    reader = csv.reader(f, skipinitialspace=True)
    #the reader object contains the iterator of the file, where each of the item contains a one-item list (i.e. every sentence is in the list)
    next(reader) #skip the headers
    # Split full comments into sentences, we use * here to flatten the list of lists
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print("Parsed %d sentences." % (len(sentences)))

# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
 
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print("Found %d unique words tokens." % len(word_freq.items()))
 
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab] # the words in vocab get the index from the list
index_to_word.append(unknown_token)
#create the dict of words and their indices in the list index_to_word
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
 
print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))
 
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
 
print("\nExample sentence: '%s'" % sentences[0])
print("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])
 
# Create the training data
#X_train contains all the word_index value in the list except the last one
#X_train[i] contains the word_to_index of words in sentence i
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
#y_train contains all the word_index value from the second word
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])
#y_train contains the id of next word for every word in X_train
print(y_train[:10])

Reading CSV file...
Parsed 79170 sentences.
Found 65752 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'bind' and appeared 10 times.

Example sentence: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'i', 'joined', 'a', 'new', 'league', 'this', 'year', 'and', 'they', 'have', 'different', 'scoring', 'rules', 'than', 'i', "'m", 'used', 'to', '.', 'SENTENCE_END']'
[ list([6, 3554, 7, 155, 797, 25, 223, 8, 32, 20, 202, 5112, 349, 91, 6, 66, 207, 5, 2, 1])
 list([11, 17, 7, 3112, 6157, 7999, 7999, 6157, 2, 1])
 list([986, 1477, 226, 595, 15, 774, 3453, 2993, 4, 7999, 595, 469, 6197, 4, 491, 595, 469, 6106, 2726, 4, 8, 71, 5732, 15, 7999, 7999, 2, 1])
 list([36, 266, 13, 4, 13, 11, 6598, 600, 12, 3063, 74, 3, 1662, 977, 15, 595, 34, 1])
 list([6, 219, 14, 3, 349, 45, 7, 3346, 650, 8, 1109, 12, 312, 4173, 86, 2405, 595

<b> Building RNN</b>
<br/>
Now we will build the RNN. The input $x$ will be a sequence of words (just like the example printed above) and each $x_t$ is a single word. But there’s one more thing: Because of how matrix multiplication works we can’t simply use a word index (like 36) as an input. Instead, we represent each word as a one-hot vector of size vocabulary_size. For example, the word with index 36 would be the vector of all 0’s and a 1 at position 36. So, each $x_t$ will become a vector, and x will be a matrix, with each row representing a word. We’ll perform this transformation in our Neural Network code instead of doing it in the pre-processing. The output of our network $o$ has a similar format. Each $o_t$ is a vector of vocabulary_size elements, and each element represents the probability of that word being the next word in the sentence.

<br/>
Let’s recap the equations for the RNN from the first part of the tutorial:

\begin{aligned}  s_t &= \tanh(Ux_t + Ws_{t-1}) \\  o_t &= \mathrm{softmax}(Vs_t)  \end{aligned}  

I always find it useful to write down the dimensions of the matrices and vectors. Let’s assume we pick a vocabulary size C = 8000 and a hidden layer size H = 100. You can think of the hidden layer size as the “memory” of our network. Making it bigger allows us to learn more complex patterns, but also results in additional computation. Then we have:

\begin{aligned}  x_t & \in \mathbb{R}^{8000} \\  o_t & \in \mathbb{R}^{8000} \\  s_t & \in \mathbb{R}^{100} \\  U & \in \mathbb{R}^{100 \times 8000} \\  V & \in \mathbb{R}^{8000 \times 100} \\  W & \in \mathbb{R}^{100 \times 100} \\  \end{aligned}  

This is valuable information. Remember that $U,V and W$ are the parameters of our network we want to learn from data. Thus, we need to learn a total of $2HC + H^2$ parameters. In the case of C=8000 and H=100 that’s 1,610,000. The dimensions also tell us the bottleneck of our model. Note that because $x_t$ is a one-hot vector, multiplying it with $U$ is essentially the same as selecting a column of $U$, so we don’t need to perform the full multiplication. Then, the biggest matrix multiplication in our network is $V*s_t$. That’s why we want to keep our vocabulary size small if possible.

Armed with this, it’s time to start our implementation.

<br/>
<b> Initialization </b>
<br/>
Initializing the parameters $U,V \space and \space W$ is a bit tricky. We can’t just initialize them to 0’s because that would result in symmetric calculations in all our layers. We must initialize them randomly. Because proper initialization seems to have an impact on training results there has been lot of research in this area. It turns out that the best initialization depends on the activation function ($\tanh$ in our case) and one recommended approach is to initialize the weights randomly in the interval from $\left[-\frac{1}{\sqrt{n}}, \frac{1}{\sqrt{n}}\right]$ where n is the number of incoming connections from the previous layer. This may sound overly complicated, but don’t worry too much it. As long as you initialize your parameters to small random values it typically works out fine.

In [3]:
class RNNNumpy:
     
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))


Above, word_dim is the size of our vocabulary, and hidden_dim is the size of our hidden layer (we can pick it). Don’t worry about the bptt_truncate parameter for now, we’ll explain what that is later.

<br/>
<b> Forward Propagation </b>
<br/>
Next, let’s implement the forward propagation (predicting word probabilities) defined by our equations above:

In [4]:
def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]
 
RNNNumpy.forward_propagation = forward_propagation

We not only return the calculated outputs, but also the hidden states. We will use them later to calculate the gradients, and by returning them here we avoid duplicate computation. Each o_t is a vector of probabilities representing the words in our vocabulary, but sometimes, for example when evaluating our model, all we want is the next word with the highest probability. We call this function predict:

In [5]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)
 
RNNNumpy.predict = predict

Let’s try our newly implemented methods and see an example output:

In [6]:
#np.random.seed(10)
#lets create a model
model = RNNNumpy(vocabulary_size)
#lets predict the probability of words from sentence 10
o, s = model.forward_propagation(X_train[10])
print(o.shape)
print(o)

(45, 8000)
[[ 0.00012505  0.00012483  0.00012451 ...,  0.00012544  0.0001246
   0.00012463]
 [ 0.00012408  0.00012509  0.00012575 ...,  0.0001254   0.00012454
   0.00012534]
 [ 0.00012521  0.00012485  0.00012505 ...,  0.00012467  0.00012498
   0.00012513]
 ..., 
 [ 0.00012547  0.00012489  0.00012531 ...,  0.00012533  0.0001244
   0.0001253 ]
 [ 0.00012551  0.00012453  0.00012654 ...,  0.00012537  0.00012433
   0.00012528]
 [ 0.00012509  0.00012589  0.00012407 ...,  0.00012611  0.00012584
   0.00012521]]


For each word in the sentence (45 above), our model made 8000 predictions representing probabilities of the next word. Note that because we initialized U,V,W to random values these predictions are completely random right now. The following gives the indices of the highest probability predictions for each word:

In [7]:
predictions = model.predict(X_train[10])
print(predictions.shape)
print(predictions) #for every word in the input sentence, find the most probable next word

(45,)
[7110 4127 4884  313 3907 6782 2438 3660 6743 2409 5717 7981 5923 5980  255
 4310 1411 1335 3850 7666 2738 4029 1681 7972 7394 7300 3304 5981 3116 7629
 5297 5279  648 7666 3301  868 7522 5980 4158  740 1083 6016 2761 1359 7987]


<b> Calculating Loss</b>
<br/>
To train our network we need a way to measure the errors it makes. We call this the loss function L, and our goal is find the parameters U,V and W that minimize the loss function for our training data. A common choice for the loss function is the cross-entropy loss. If we have N training examples (words in our text) and C classes (the size of our vocabulary) then the loss with respect to our predictions o and the true labels y is given by:

\begin{aligned}  L(y,o) = - \frac{1}{N} \sum_{n \in N} y_{n} \log o_{n}  \end{aligned}  

The formula looks a bit complicated, but all it really does is sum over our training examples and add to the loss based on how off our prediction are. The further away y (the correct words) and o (our predictions), the greater the loss will be. We implement the function calculate_loss:

In [8]:
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        #in the output array (o), check the index of predictions made for each word
        # in the sentence i.
        #print(y[i])
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        #print("correct word predictions is:")
        #print(correct_word_predictions)
        # Add to the loss based on how off we were
        #ALERT! ALERT!! As per resource [ref 1], the ideal output has predictions of 1
        # for all the words. Ideally, it assumes every word can be the next word, so
        # the probability of 1. Hence the loss function just adds how much deviation
        # has been there from this ideal case (i.e. having all 1s)
        #This is not an efficient method though
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L
 
def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    # for every sentence in y, count the number of next words and get the sum for
    # total entries in the data y
    N = np.sum((len(y_i) for y_i in y))
    #ALERT! ALERT!! is dividing word indices by total words efficient here?
    return self.calculate_total_loss(x,y)/N
 
RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

#print(X_train.shape,y_train.shape)
#print(X_train.shape, len(tokenized_sentences))
#print(y_train[0])


Let’s take a step back and think about what the loss should be for random predictions. That will give us a baseline and make sure our implementation is correct. We have C words in our vocabulary, so each word should be (on average) predicted with probability 1/C, which would yield a loss of 
$L = -\frac{1}{N} N \log\frac{1}{C} = \log C$:

In [9]:
# Limit to 1000 examples to save time
print("Expected Loss for random predictions: %f" % np.log(vocabulary_size))
print("Actual loss: %f" % model.calculate_loss(X_train[:10], y_train[:10]))

Expected Loss for random predictions: 8.987197
Actual loss: 8.987443


<b> TRAINING THE RNN WITH SGD AND BACKPROPAGATION THROUGH TIME (BPTT) </b>
<br/>
<br/>

We want to find the parameters $U,V \space and \space W$ that minimize the total loss on the training data. The most common way to do this is SGD, Stochastic Gradient Descent. The idea behind SGD is pretty simple. We iterate over all our training examples and during each iteration we nudge the parameters into a direction that reduces the error.
These directions are given by the gradients on the loss: $\frac{\partial L}{\partial U}, \frac{\partial L}{\partial V}, \frac{\partial L}{\partial W}$. SGD also needs a learning rate, which defines how big of a step we want to make in each iteration. SGD is the most popular optimization method not only for Neural Networks, but also for many other Machine Learning algorithms. As such there has been a lot of research on how to optimize SGD using batching, parallelism and adaptive learning rates. 
Even though the basic idea is simple, implementing SGD in a really efficient way can become very complex. If you want to learn more about SGD <a href="http://cs231n.github.io/optimization-1/"> this </a> is a good place to start.
Lets implement a simple version of SGD that should be understandable even without a background in optimization.

<br/>
But how do we calculate those gradients we mentioned above? In a traditional Neural Network we do this through the backpropagation algorithm. In RNNs we use a slightly modified version of the this algorithm called Backpropagation Through Time (BPTT). Because the parameters are shared by all time steps in the network, the gradient at each output depends not only on the calculations of the current time step, but also the previous time steps. If you know calculus, it really is just applying the chain rule. The next part of the tutorial will be all about BPTT. For a general introduction to backpropagation check out <a href="http://colah.github.io/posts/2015-08-Backprop/"> this </a> and <a href="http://cs231n.github.io/optimization-2/"> this</a> post. 

For now we can treat BPTT as a black box. It takes as input a training example (x,y) and returns the gradients $\frac{\partial L}{\partial U}, \frac{\partial L}{\partial V}, \frac{\partial L}{\partial W}$.

In [10]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]
 
RNNNumpy.bptt = bptt

<b> GRadient Checking</b>
<br/>
Whenever you implement backpropagation it is good idea to also implement gradient checking, which is a way of verifying that your implementation is correct. The idea behind gradient checking is that derivative of a parameter is equal to the slope at the point, which we can approximate by slightly changing the parameter and then dividing by the change:
<img src ="images/gradient_checking-rnn.png"/>
We then compare the gradient we calculated using backpropagation to the gradient we estimated with the method above. If there’s no large difference we are good. The approximation needs to calculate the total loss for every parameter, so that gradient checking is very expensive (remember, we had more than a million parameters in the example above). So it’s a good idea to perform it on a model with a smaller vocabulary.

In [11]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = self.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print("Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape)))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = self.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = self.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print("Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix))
                print("+h Loss: %f" % gradplus)
                print( "-h Loss: %f" % gradminus)
                print("Estimated_gradient: %f" % estimated_gradient)
                print("Backpropagation gradient: %f" % backprop_gradient)
                print("Relative Error: %f" % relative_error)
                return
            it.iternext()
        print("Gradient check for parameter %s passed." % (pname))
 
RNNNumpy.gradient_check = gradient_check
 
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.


C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


<b> SGD IMPLEMENTATION </b>
<br/>
<br/>
Now that we are able to calculate the gradients for our parameters we can implement SGD. I like to do this in two steps: 1. A function sdg_step that calculates the gradients and performs the updates for one batch. 2. An outer loop that iterates through the training set and adjusts the learning rate.

In [12]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW
 
RNNNumpy.sgd_step = numpy_sdg_step

# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5 
                print("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

Done! Let’s try to get a sense of how long it would take to train our network:

In [13]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

1 loop, best of 3: 293 ms per loop


One step of SGD takes approximately 350 milliseconds in a normal machine. We have about 80,000 examples in our training data, so one epoch (iteration over the whole data set) would take several hours. Multiple epochs would take days, or even weeks! And we’re still working with a small dataset compared to what’s being used by many of the companies and researchers out there. What now?

There are many ways to speed up our code. We could stick with the same model and make our code run faster, or we could modify our model to be less computationally expensive, or both. Researchers have identified many ways to make models less computationally expensive, for example by using a hierarchical softmax or adding projection layers to avoid the large matrix multiplications (see also <a href="http://arxiv.org/pdf/1301.3781.pdf"> here </a> or <a href="http://www.fit.vutbr.cz/research/groups/speech/publi/2011/mikolov_icassp2011_5528.pdf"> here </a>). But I want to keep our model simple and go the first route: Make our implementation run faster using a GPU. Before doing that though, let’s just try to run SGD with a small dataset and check if the loss actually decreases:

In [14]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNNumpy(vocabulary_size)
losses = train_with_sgd(model, X_train[:100], y_train[:100], nepoch=10, evaluate_loss_after=1)

2017-06-16 10:46:57: Loss after num_examples_seen=0 epoch=0: 8.987484
2017-06-16 10:47:16: Loss after num_examples_seen=100 epoch=1: 8.976138
2017-06-16 10:47:36: Loss after num_examples_seen=200 epoch=2: 8.959782
2017-06-16 10:47:55: Loss after num_examples_seen=300 epoch=3: 8.929152
2017-06-16 10:48:13: Loss after num_examples_seen=400 epoch=4: 8.809424
2017-06-16 10:48:31: Loss after num_examples_seen=500 epoch=5: 6.701028
2017-06-16 10:48:52: Loss after num_examples_seen=600 epoch=6: 6.223796
2017-06-16 10:49:10: Loss after num_examples_seen=700 epoch=7: 5.975018
2017-06-16 10:49:29: Loss after num_examples_seen=800 epoch=8: 5.807733
2017-06-16 10:49:47: Loss after num_examples_seen=900 epoch=9: 5.690627


Good, it seems like our implementation is at least doing something useful and decreasing the loss, just like we wanted.

<br/>
<b> TRAINING OUR NETWORK WITH THEANO AND THE GPU </b>


In [20]:
#import sys
#reload(sys)
#sys.setdefaultencoding('utf8')
#import the file that has Theano related methods, I have it saved inside rnn_theano.py
from rnn_theano import RNNTheano
np.random.seed(10)
model = RNNTheano(vocabulary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

10 loops, best of 3: 150 ms per loop


<b> Using pretrained model</b>
<br/>
To help you avoid spending days training a model I have pre-trained a Theano model with a hidden layer dimensionality of 50 and a vocabulary size of 8000. I trained it for 50 epochs in about 20 hours. The loss was was still decreasing and training longer would probably have resulted in a better model, but I was running out of time and wanted to publish this post. Feel free to try it out yourself and trian for longer

In [21]:
from rnn_utils import load_model_parameters_theano, save_model_parameters_theano
 
model = RNNTheano(vocabulary_size, hidden_dim=50)
# losses = train_with_sgd(model, X_train, y_train, nepoch=50)
# save_model_parameters_theano('./data/trained-model-theano.npz', model)
load_model_parameters_theano('data/trained-model-theano.npz', model)

Loaded model parameters from data/trained-model-theano.npz. hidden_dim=50 word_dim=8000


<b>GENERATING TEXT</b>
<br/>
Now that we have our model we can ask it to generate new text for us! Let’s implement a helper function to generate new sentences:

In [22]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str
 
num_sentences = 10
senten_min_length = 7
 
for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print(" ".join(sent))

no to location their stuff go at all .
characters fault number fighting but services every game .
no watch work on the macro signs a supreme mecha .
the ^creator what is comments hard .
me do you effect bad doubled .
*is* is much good , no .
me so many different sitting modern .
probably not very a bot or turns .
correct this is wash so why ?
tds but a ruin gun afraid .


In [23]:
'''
import tensorflow as tf
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)
sess = tf.Session()
print(sess.run([node1, node2]))
'''
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[Elemwise{exp,no_inplace}(<TensorType(float64, vector)>)]
Looping 1000 times took 18.928613 seconds
Result is [ 1.23178032  1.61879341  1.52278065 ...,  2.20771815  2.29967753
  1.62323285]
Used the cpu


<b> References </b>
<br/>
<ol>
<li> <a href="http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/"> Implementing RNN using Python and Theano</a>

<li> <a href="http://www.wildml.com/2015/10/recurrent-neural-networks-tutorial-part-3-backpropagation-through-time-and-vanishing-gradients/"> Understanding Backpropagation trhough time (BPTT) algorithm and vanishing gradient problem </a>

<li> <a href="http://www.wildml.com/2015/10/recurrent-neural-network-tutorial-part-4-implementing-a-grulstm-rnn-with-python-and-theano/">Implementing a GRU/LSTM RNN </a>

<li> <a href="http://www.fit.vutbr.cz/research/groups/speech/publi/2010/mikolov_interspeech2010_IS100722.pdf"> Recurrent neural network based language model from Mikolov et al. </a>

<li> <a href="http://karpathy.github.io/2015/05/21/rnn-effectiveness/"> RNN effectiveness </a>
</ol>